In [12]:
import pandas as pd
## for Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
# from read_data import load_df
import numpy as np
from preprocessing import PrepareData
from jupyterthemes import jtplot
jtplot.style()
from keras.layers import TimeDistributed

from keras.callbacks import ReduceLROnPlateau
import datetime
from keras.callbacks import TensorBoard

In [2]:
def shuffle(X,Y):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

In [3]:
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

In [4]:
def buildTrain(train, pastDay=5, futureDay=1):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["SalePrice"]))
    return np.array(X_train), np.array(Y_train)

In [36]:
def buildModel(shape):
    model = Sequential()
    model.add(LSTM(50, input_shape=(shape[1], shape[2]), return_sequences=False))
#     model.add(Dropout(0.2))
    model.add(Dense(1, activation="tanh"))
    model.compile(loss="mean_squared_error", optimizer="rmsprop")
    model.summary()
    return model

In [37]:
batch_size = 4
ppd = PrepareData()
df , sale_price = ppd.get_train_data(time_series=True, orderby='YearBuilt')
# df = df.sort_index()
X_train, Y_train = buildTrain(df, pastDay=batch_size)

In [42]:
pd.concat([df, sale_price], axis=1)

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave,Utilities_AllPub,Utilities_NoSeWa,SalePrice,SalePrice
dt,,,,,,,,,,,,,,,,,,,,,
2006-01-01,0.195205,0.041319,0.555556,0.625,0.739130,0.816667,0.061875,0.333333,0.50,0.333333,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.202888,181000
2006-01-01,0.167979,0.043745,0.444444,0.625,0.644928,0.183333,0.000000,0.333333,0.50,0.333333,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.141092,136500
2006-01-01,0.095890,0.091594,0.777778,0.500,0.963768,0.933333,0.000000,0.666667,0.50,1.000000,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.538953,423000
2006-01-01,0.332192,0.057978,0.666667,0.500,0.934783,0.866667,0.093750,0.666667,0.50,0.666667,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.236217,205000
2006-01-01,0.195205,0.048354,0.666667,0.500,0.963768,0.916667,0.000000,0.666667,0.50,0.666667,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.312595,260000
2006-01-01,0.270548,0.155881,0.444444,0.500,0.601449,0.083333,0.000000,0.333333,0.50,0.333333,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.152895,145000
2006-01-01,0.167979,0.071836,0.777778,0.625,0.789855,0.516667,0.408125,0.666667,0.50,0.666667,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.268157,228000
2006-01-01,0.229452,0.055271,0.444444,0.375,0.615942,0.833333,0.041875,0.333333,0.50,0.333333,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.166782,155000
2006-01-01,0.167979,0.031882,0.555556,0.500,0.927536,0.833333,0.000000,0.333333,0.50,0.666667,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.190946,172400


In [38]:
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)
print X_train.shape
print Y_train.shape
print X_val.shape
print Y_val.shape

(1310, 4, 246)
(1310, 1)
(145, 4, 246)
(145, 1)


In [41]:
model = buildModel(X_train.shape)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)
early_stop = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
tf_board = TensorBoard(log_dir='./logs/{}'.format(datetime.datetime.now().strftime('%H_%M')), histogram_freq=0, batch_size=batch_size, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

batch_size =1
nb_epoch = 50
for i in range(nb_epoch):
    model.fit(X_train, Y_train, epochs=1, batch_size=batch_size, shuffle=False, validation_data=(X_val, Y_val), callbacks=[tf_board, reduce_lr, early_stop])
    model.reset_states()
# history = model.fit(
#     X_train, Y_train, epochs=70, validation_data=(X_val, Y_val), callbacks=[tf_board, reduce_lr, early_stop])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 50)                59400     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 59,451
Trainable params: 59,451
Non-trainable params: 0
_________________________________________________________________
Train on 1310 samples, validate on 145 samples
Epoch 1/1
1310/1310 [==============================] - 6s 4ms/step - loss: 0.0162 - val_loss: 0.0138
Train on 1310 samples, validate on 145 samples
Epoch 1/1
1310/1310 [==============================] - 5s 4ms/step - loss: 0.0125 - val_loss: 0.0145
Train on 1310 samples, validate on 145 samples
Epoch 1/1
1310/1310 [==============================] - 5s 4ms/step - loss: 0.0121 - val_loss: 0.0152
Train on 1310 samples, validate on 145 samples
Epoch 1/1
1310/1310 [======================

In [ ]:
# make a prediction
pred_train = model.predict(X_train)
print 'RMSE (Training Data):', mean_squared_error(Y_train , pred_train)
pred_test = model.predict(X_val)
print 'RMSE (Testin Data):', mean_squared_error(Y_val , pred_test)

# show distribution
plt.scatter(pred_test, Y_val, alpha=.75,
            color='b') #alpha helps to show overlapping data
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('LSTM mumtiple timestamps')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
model.reset_states()

### Hyper-parameters tunning

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_network(shape, optimizer='rmsprop'):
    model = Sequential()
    model.add(LSTM(600, input_shape=(shape[1], shape[2]), return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(600, input_shape=(shape[1], shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
    return model

# Wrap Keras model so it can be used by scikit-learn
sk_params = {
    'shape': X_train.shape
}
neural_network = KerasClassifier(build_fn=create_network, verbose=0, **sk_params)
# Create hyperparameter space
epochs = [5, 10]
batches = [5, 10, 100]
optimizers = ['rmsprop', 'adam']

# Create hyperparameter options
hyperparameters = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)
grid = GridSearchCV(estimator=neural_network, param_grid=hyperparameters)

# Fit grid search
grid_result = grid.fit(X_train, Y_train)
print grid_result.best_params_
print grid_result.best_score_